- 이것이 데이터 분석이다 책 참고
  - 3.2) 비트코인 시세 예측하기
  - https://github.com/yoonkt200/python-data-analysis/blob/master/chapter3/02-coin-prediction.ipynb

In [8]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [16]:
file_path = 'data/train.csv'
df = pd.read_csv(file_path)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3279 entries, 0 to 3278
Data columns (total 37 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   날짜      3279 non-null   int64
 1   시간      3279 non-null   int64
 2   10      3279 non-null   int64
 3   100     3279 non-null   int64
 4   101     3279 non-null   int64
 5   120     3279 non-null   int64
 6   121     3279 non-null   int64
 7   140     3279 non-null   int64
 8   150     3279 non-null   int64
 9   160     3279 non-null   int64
 10  200     3279 non-null   int64
 11  201     3279 non-null   int64
 12  251     3279 non-null   int64
 13  270     3279 non-null   int64
 14  300     3279 non-null   int64
 15  301     3279 non-null   int64
 16  351     3279 non-null   int64
 17  352     3279 non-null   int64
 18  370     3279 non-null   int64
 19  400     3279 non-null   int64
 20  450     3279 non-null   int64
 21  500     3279 non-null   int64
 22  550     3279 non-null   int64
 23  600     3279 

In [13]:
df.tail()

,날짜,시간,10,100,101,120,121,140,150,160,...,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
3274,20200517,19,311727,101285,10085,30637,10060,8749,148935,6801,...,6726,15431,25597,14292,9300,22238,3786,16936,10729,20194
3275,20200517,20,305354,91426,8607,26021,8095,7198,136503,6147,...,5501,15378,24661,14747,8239,20604,3203,15018,9767,17962
3276,20200517,21,306008,75113,6325,19933,5711,4494,129412,5134,...,4216,12558,22781,14081,6392,17937,2447,12403,7825,14031
3277,20200517,22,237447,49498,4209,12145,3891,2718,96698,3526,...,2578,8870,16640,11066,4427,11955,1495,7507,5387,8889
3278,20200517,23,150312,27410,2350,6406,1803,1614,55788,1849,...,1377,5021,10058,7139,2250,6844,735,4116,3046,4606


In [20]:
# to_datetime으로 day 피처를 시계열 피처로 변환합니다. 
df['day'] = df['날짜'].astype(str)
df['day'] = pd.to_datetime(df['day'])

# day 데이터프레임의 index로 설정합니다.
df.index = df['day']
df.set_index('day', inplace=True)
df.head()

,날짜,시간,10,100,101,120,121,140,150,160,...,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
day,,,,,,,,,,,,,,,,,,,,,
2020-01-01,20200101,0,83247,19128,2611,5161,1588,892,32263,1636,...,1311,3482,11299,7072,1176,3810,748,3920,2133,3799
2020-01-01,20200101,1,89309,19027,3337,5502,1650,1043,35609,1644,...,1162,3849,13180,8771,1283,3763,782,3483,2057,4010
2020-01-01,20200101,2,66611,14710,2970,4631,1044,921,26821,1104,...,768,2299,7986,5426,1536,3229,491,2634,1526,3388
2020-01-01,20200101,3,53290,13753,2270,4242,1021,790,21322,909,...,632,1716,5703,3156,1104,2882,431,2488,1268,3686
2020-01-01,20200101,4,52095,17615,2406,3689,1840,922,22711,1354,...,875,2421,5816,2933,1206,2433,499,2952,1927,5608


In [42]:
df1 = df.copy()
df1 = df1.drop(['날짜','시간'], axis=1)

# [Facebook Prophet 활용하기]

- prophet의 input data columns는 'ds'와 'y'로 고정되어야 함
- seasonality_mode : 연간, 월간, 주간, 일간 등의 트렌드성을 반영하는 것을 의미
- hangepoint_prior_scale : 트렌드가 변경되는 문맥을 반영하여 예측. 수치가 높을수록 overfitting에 가까워짐.

In [ ]:
from fbprophet import Prophet

In [7]:
# prophet을 사용하기 위해서는 다음과 같이 피처의 이름을 변경해야 함 : 'ds', 'y'
df = pd.read_csv(file_path, names=['ds', 'y'])

prophet = Prophet(seasonality_mode='multiplicative', 
                  yearly_seasonality=True,
                  weekly_seasonality=True, daily_seasonality=True,
                  changepoint_prior_scale=0.5)

prophet.fit(df)